In [1]:
from model import train_crm
from huggingface_hub import hf_hub_download
import json
from util.utils import get_tri
from PIL import Image
import nvdiffrast.torch as dr
import torch
import tqdm
import numpy as np
import kaolin as kal
from matplotlib import pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
import render
import loss
from util_flexi import *

In [2]:
iter = 1000
batch = 8
train_res = [1024, 1024]
learning_rate = 0.01
voxel_grid_res = 64
device = 'cuda'
sdf_regularizer = 0.2

In [3]:

crm_path = hf_hub_download(repo_id="Zhengyi/CRM", filename="CRM.pth")
specs = json.load(open("configs/specs_objaverse_total.json"))

model_crm = train_crm(specs).to('cuda:0')
model_crm.load_state_dict(torch.load(crm_path, map_location = device), strict=False)
# model_crm.load_state_dict(torch.load('model_weights_final2.pth', map_location = device), strict=False)

fc = model_crm.renderer.flexicubes
x_nx3 = fc.verts
sdf = torch.rand_like(x_nx3[:,0]) - 0.1
sdf = torch.nn.Parameter(sdf.clone().detach(), requires_grad=True)

grid_edges = fc.all_edges


gt_mesh = kal.io.obj.import_mesh('GT_shoe/gtShoe_1.obj').cuda()
vertices = gt_mesh.vertices
vmin, vmax = vertices.min(dim=0)[0], vertices.max(dim=0)[0]
scale = 1.8 / torch.max(vmax - vmin).item()
vertices = vertices - (vmax + vmin) / 2 # Center mesh on origin
gt_mesh.vertices = vertices * scale # Rescale to [-0.9, 0.9]


# fc = kal.non_commercial.FlexiCubes(device)
# x_nx3, cube_fx8 = fc.construct_voxel_grid(voxel_grid_res)
# x_nx3 *= 2 # scale up the grid so that it's larger than the target object
# sdf = torch.rand_like(x_nx3[:,0]) - 0.1 # randomly initialize SDF
# sdf = torch.nn.Parameter(sdf.clone().detach(), requires_grad=True)
# # set per-cube learnable weights to zeros
# weight = torch.zeros((cube_fx8.shape[0], 21), dtype=torch.float, device='cuda') 
# weight = torch.nn.Parameter(weight.clone().detach(), requires_grad=True)

# #  Retrieve all the edges of the voxel grid; these edges will be utilized to 
# #  compute the regularization loss in subsequent steps of the process.
# all_edges = cube_fx8[:, fc.cube_edges].reshape(-1, 2) 
# grid_edges = torch.unique(all_edges, dim=0)
# print(model_crm.parameters())
# Define optimizer




In [4]:
def data_loader(data_num:int):
    rgb=[]
    for i in range(6):
            img = Image.open(f'shoe{data_num}_processed/1{i}.png')

            new_width  = 256
            new_height = new_width 
            new_height = 256
            new_width  = new_height 
            pixel_img = img.resize((new_width, new_height), Image.LANCZOS)

            # print(pixel_img.size)
            arr = np.asarray(pixel_img)
            rgb.append(arr)

    rgb = [Image.fromarray(img) for img in rgb]
    ccm = [Image.open(f'shoe{data_num}_normals/{i}.png') for i in range(6)]
    np_imgs = np.concatenate(rgb, 1)
    np_xyzs = np.concatenate(ccm, 1)
    # gt_sdf = get_gt_sdf(mesh1_sdf)

    return(np_imgs,np_xyzs)

In [5]:
def lr_schedule(iter):
    return max(0.0, 10 ** (-(iter) * 0.0002)) # Exponential falloff from [1.0, 0.1] over 5k epochs.   
# Define optimizer 
optimizer = optim.Adam(model_crm.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda x: lr_schedule(x)) 

In [6]:
def generate_3d(rgb, ccm,model):
    color_tri = torch.from_numpy(rgb)/255
    xyz_tri = torch.from_numpy(ccm[:,:,(2,1,0)])/255
    color = color_tri.permute(2,0,1)
    xyz = xyz_tri.permute(2,0,1)


    def get_imgs(color):
        # color : [C, H, W*6]
        color_list = []
        color_list.append(color[:,:,256*5:256*(1+5)])
        for i in range(0,5):
            color_list.append(color[:,:,256*i:256*(1+i)])
        return torch.stack(color_list, dim=0)# [6, C, H, W]

    triplane_color = get_imgs(color).permute(0,2,3,1).unsqueeze(0).to(device)# [1, 6, H, W, C]

    color = get_imgs(color)
    xyz = get_imgs(xyz)

    color = get_tri(color, dim=0, blender= True, scale = 1).unsqueeze(0)
    xyz = get_tri(xyz, dim=0, blender= True, scale = 1, fix= True).unsqueeze(0)

    triplane = torch.cat([color,xyz],dim=1).to(device)
    # 3D visualize
    model.eval()
    glctx = dr.RasterizeCudaContext()

    if model.denoising == True:
        tnew = 20
        tnew = torch.randint(tnew, tnew+1, [triplane.shape[0]], dtype=torch.long, device=triplane.device)
        noise_new = torch.randn_like(triplane) *0.5+0.5
        triplane = model.scheduler.add_noise(triplane, noise_new, tnew)    
        # start_time = time.time()
        with torch.no_grad():
            triplane_feature2 = model.unet2(triplane,tnew)
        # end_time = time.time()
        # elapsed_time = end_time - start_time
        # print(f"unet takes {elapsed_time}s")
    else:
        triplane_feature2 = model.unet2(triplane)
        

    with torch.no_grad():
        data_config = {
            'resolution': [1024, 1024],
            "triview_color": triplane_color.to(device),
        }

        result,vert,faces,weight = model.decode(data_config, triplane_feature2)
        # print(data[1][0])
        data_config['verts'] = vert[0]
        data_config['faces'] = faces
        data_config =  data_config | result
        data_config['weight'] = weight
    return data_config

In [7]:
rgb,ccm = data_loader(1)
data_config = generate_3d(rgb,ccm,model_crm)
flexicubes_mesh = kal.rep.SurfaceMesh(vertices=data_config['verts'], faces=data_config['faces'].to(torch.int64))
intermediate_results = [flexicubes_mesh]

In [8]:
loop = tqdm.tqdm(range(iter),leave=True)
for it in loop: 

    optimizer.zero_grad()
    # sample random camera poses
    cameras = render.get_random_camera_batch(batch, iter_res=train_res, device=device)
    
    # render gt mesh at sampled views
    target = render.render_mesh(gt_mesh, cameras, train_res)

    data_config = generate_3d(rgb,ccm,model_crm)
    
    flexicubes_mesh = kal.rep.SurfaceMesh(vertices=data_config['verts'], faces=data_config['faces'].to(torch.int64))

    buffers = render.render_mesh(flexicubes_mesh, cameras, train_res)

    # evaluate reconstruction loss
    mask_loss = (buffers['mask'] - target['mask']).abs().mean() # mask loss
    depth_loss = (((((buffers['depth'] - (target['depth']))* target['mask'])**2).sum(-1)+1e-8)).sqrt().mean() * 10 # depth loss


    # evaluate regularization losses
    t_iter = it / iter
    # this is the regularization loss described in Equation 2 of the nvdiffrec paper by Munkberg et al., which serves to remove internal floating elements that are not visible to the user.
    sdf_weight = sdf_regularizer - (sdf_regularizer - sdf_regularizer/20)*min(1.0, 4.0 * t_iter)
    reg_loss = loss.sdf_reg_loss(sdf, grid_edges).mean() * sdf_weight

    reg_loss += data_config["flex_surf_loss"]*0.5 # L_dev as in Equation 8 of our paper
    reg_loss += (data_config["weight"][:,:20]).abs().mean() * 0.1  # regularize weights to be zeros to improve the stability of the optimization process
    total_loss = mask_loss + depth_loss + reg_loss
    

    if True: # optionally add SDF loss to eliminate internal structures
            with torch.no_grad():
                pts = sample_random_points(1000, gt_mesh)
                gt_sdf = compute_sdf(pts, gt_mesh.vertices, gt_mesh.faces)
            pred_sdf = compute_sdf(pts, flexicubes_mesh.vertices, flexicubes_mesh.faces)
            total_loss += torch.nn.functional.mse_loss(pred_sdf, gt_sdf) * 2e3
        
    # optionally add developability regularizer, as described in paper section 5.2
    # if True:
    #     reg_weight = max(0, t_iter - 0.8) * 5
    #     if reg_weight > 0: # only applied after shape converges
    #         reg_loss = loss.mesh_developable_reg(flexicubes_mesh).mean() * 10
    #         reg_loss += (deform).abs().mean()
    #         reg_loss += (weight[:,:20]).abs().mean()
    #         total_loss = mask_loss + depth_loss + reg_loss 
            
    total_loss.backward()
    optimizer.step()
    scheduler.step()
    if it % 100 == 0:
            log=open('log.txt','a')
            log.write(f'epoch = {it} : loss = {total_loss.item()}\n')
            log.close()

    if it % 1000 == 0:
        log=open('log.txt','a')
        torch.save(model_crm.state_dict(), f'checkpoints/model_weights_1_2_{it}.pth')
        log.write(f'[Saved checkpoint] epoch = {it} : loss = {total_loss.item()}\n')
        log.close()

    if (it + 1) % 20 == 0: # save intermediate results every 100 iters
        with torch.no_grad():
            # run the mesh extraction again with the parameter 'training=False' so that each quadrilateral face is divided into two triangles, as opposed to the four triangles during the training phase.
            data_config = generate_3d(rgb,ccm,model_crm)
            intermediate_results.append(kal.rep.SurfaceMesh(vertices=data_config['verts'], faces=data_config['faces'].to(torch.int64)))

    # update tqdm loop
    loop.set_postfix(loss = total_loss.item())




100%|██████████| 1000/1000 [1:12:03<00:00,  4.32s/it, loss=0.522]


In [9]:
torch.save(model_crm.state_dict(), 'model_weights_final2_lrSH.pth')

In [10]:
camera = render.get_rotate_camera(0)
render.TimelineVisualizer(intermediate_results, 512, 512).show(camera)

Output()